### 9.1 Wrapper

In [1]:
import time
import types
import operator

In [2]:
from functools import wraps
from inspect import signature
from collections import OrderedDict

In [3]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [4]:
@timeit
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
        
countdown(100000)
countdown(1000)
        

countdown 0.008379936218261719
countdown 0.00010585784912109375


In [5]:
countdown.__name__, countdown.__doc__, countdown.__annotations__, countdown.__wrapped__

('countdown',
 '\n    Counts down\n    ',
 {'n': int},
 <function __main__.countdown(n: int)>)

In [6]:
print(signature(countdown))

(n: int)


### 9.4 Decorator with arguments

In [7]:
import logging
logging.basicConfig(level=logging.WARNING)

In [8]:
from functools import partial

# Utility decorator to attach a func as an attribute of an obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj) # This is the wrapper around the func -> attach_wrapper(obj)(func)
    setattr(obj, func.__name__, func)
    return func

In [9]:
def logged(level, name=None, message=None):
    
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
    
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
        
    return decorate

In [10]:
@logged(logging.WARNING)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [11]:
spam()
add(3, 4)

CRITICAL:example:spam


Spam!


7

In [12]:
spam.set_level(logging.ERROR)
spam.set_message('Custom msg')
spam()

ERROR:example:Custom msg


Spam!


### 9.7 Type Assertion

In [13]:
def spam(x, y, z=42):
    pass

In [14]:
sig = signature(spam)
print(sig)

(x, y, z=42)


In [15]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [16]:
sig.parameters['z'].name, sig.parameters['z'].default, sig.parameters['z'].kind

('z', 42, <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>)

In [17]:
bound_types = sig.bind_partial(int, z=int)
bound_types.arguments

OrderedDict([('x', int), ('z', int)])

In [18]:
def typeassert(*ty_args, **ty_kwargs):
    
    def decorate(func):
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs).arguments
            # Enforce type assertions
            for name, value in bound_values.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError('Argument {} must be of type {}'.format(name, bound_types[name]))
            return func(*args, **kwargs)
        return wrapper
    return decorate
            
            

In [19]:
@typeassert(int, int)
def add(x, y=4):
    return x + y

@typeassert(y=str)
def printer(x, y, z):
    print(x, y, z)

print('2 + 3 = {}'.format(add(2, 3)))
printer(1, 'world', 3)
try:
    add('bat', 'man')
except TypeError as err:
    print('Error: {}'.format(err))
    
try:
    printer(1, 2, 3)
except TypeError as err:
    print('Error: {}'.format(err))

2 + 3 = 5
1 world 3
Error: Argument x must be of type <class 'int'>
Error: Argument y must be of type <class 'str'>


### 9.9 Decorator as a Class

In [20]:
class Profiled:
    
    def __init__(self, func):
        print('__init__ called')
        wraps(func)(self)
        self.ncalls = 0
        
    def __call__(self, *args, **kwargs):
        print('__call__ called')
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
    
    def __get__(self, instance, cls):
        print('__get__ called')
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)
    

In [21]:
@Profiled
def sub(x, y):
    return x-y

__init__ called


In [22]:
print(sub(20,3))
sub.ncalls

__call__ called
17


1

In [23]:
class Spam:
    @Profiled
    def bar(self, x):
        print(x)
    
    def foo(self, y):
        print(y)

__init__ called


In [24]:
s = Spam()
s.bar('author')
s.bar('again')
s.bar.ncalls

__get__ called
__call__ called
author
__get__ called
__call__ called
again
__get__ called


2

In [25]:
def profiled(func):
    ncalls = 0
    @wraps(func)
    def wrapped(*args, **kwargs):
        nonlocal ncalls
        ncalls += 1
        return func(*args, **kwargs)
    wrapped.ncalls = lambda : ncalls
    return wrapped

In [26]:
@profiled
def mul(x, y):
    return x*y

mul(2, 3)
mul(3, 4)
mul.ncalls()

2

### 9.11 Decorators to add arguments to functions

In [27]:
def optional_debug(func):
    @wraps(func)
    def wrapped(*args, debug=False, **kwargs):
        if debug:
            print('Calling {}'.format(func.__name__))
        return func(*args, **kwargs)
    return wrapped

In [28]:
@optional_debug
def spam(x, y, z):
    print(x, y, z)

In [29]:
spam(1, 2, 3)

1 2 3


In [30]:
spam(3, 4, 5, debug=True)

Calling spam
3 4 5


### 9.13 MetaClass - Class Creation

In [31]:
class NoInstances(type):
    def __call__(cls, *args, **kwargs):
        raise RuntimeError('Can not instantiate {}'.format(cls))
        
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')
        

In [32]:
Spam.grok('author')
try:
    s = Spam('Guido')
except RuntimeError as err:
    print(err)

Spam.grok
Can not instantiate <class '__main__.Spam'>


In [33]:
class Singleton(type):
    def __init__(cls, name, bases, dct):
        cls.__instance = None
        super().__init__(name, bases, dct)
        
    def __call__(cls, *args, **kwargs):
        if cls.__instance is None:
            cls.__instance =  super().__call__(*args, **kwargs)
        return cls.__instance
        
            
class SingleSpam(metaclass=Singleton):
    def __init__(self):
        print('Creating SingleSpam')

In [34]:
s = SingleSpam()
q = SingleSpam()
s is q

Creating SingleSpam


True

### 9.14 Class Attribute Definition Order

In [35]:
class Typed:
    _expected_type = None
    def __init__(self, name=None):
        self._name = name
        
    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected {!s} for {}'.format(self._expected_type, self._name))
        instance.__dict__[self._name] = value
        
        

In [36]:
class Integer(Typed):
    _expected_type = int
    
class Float(Typed):
    _expected_type = float
    
class String(Typed):
    _expected_type = str

In [37]:
class OrderedMeta(type):
    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()
    
    def __new__(meta, clsname, bases, clsdct):
        d = dict(clsdct)
        order = []
        for name, value in clsdct.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return super().__new__(meta, clsname, bases, d)
                

In [38]:
class Structure(metaclass=OrderedMeta):
    def to_csv(self):
        return ','.join(str(getattr(self, name)) for name in self._order)
    
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [39]:
try:
    goog = Stock('GOOG', 10, 1500)
except TypeError as err:
    print(err)
amazon = Stock('AMAZN', 10, 2000.)
amazon.to_csv()

Expected <class 'float'> for price


'AMAZN,10,2000.0'

### 9.16 Enforcing argument signature

In [40]:
import inspect
from inspect import Signature, Parameter

In [41]:
temp_parms = [ Parameter('x', Parameter.POSITIONAL_OR_KEYWORD), 
               Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42), 
               Parameter('z', Parameter.KEYWORD_ONLY, default=None) ]
temp_sig = Signature(temp_parms)
print(temp_sig)

(x, y=42, *, z=None)


In [42]:
def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in names]
    return Signature(parms)

In [43]:
class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values:
            setattr(self, name, value)
            
            
class Point(Structure):
    __signature__ = make_sig('x', 'y')

In [44]:
print(inspect.signature(Point))

(x, y)


### 9.17 Coding Conventions in Classes

In [45]:
class NoMixedCaseMeta(type):
    def __new__(meta, clsname, bases, clsdct):
        for name in clsdct:
            if name.lower() != name:
                raise TypeError('Bad attribute name: {}'.format(name))
        return super().__new__(meta, clsname, bases, clsdct)    

In [46]:
class Root(metaclass=NoMixedCaseMeta):
    pass

class A(Root):
    def foo_bar(self):
        pass

In [47]:
try:
    class B(Root):
        def FooBar(self):
            pass
except TypeError as err:
    print(err)

Bad attribute name: FooBar


In [48]:
class MatchSignaturesMeta(type):
    def __init__(cls, clsname, bases, clsdct):
        super().__init__(clsname, bases, clsdct)
        sup = super(cls, cls)
        for name, value in clsdct.items():
            if name.startswith('_') or not callable(value):
                continue
            prev_dfn = getattr(sup, name, None)
            if prev_dfn:
                prev_sig = inspect.signature(prev_dfn)
                curr_sig = inspect.signature(value)
                if prev_sig != curr_sig:
                    logging.warning('Signature mismatch in {}. {} != {}'.format(value.__qualname__, curr_sig, prev_sig))
                   

In [49]:
class Root(metaclass=MatchSignaturesMeta):
    pass

class A(Root):
    def foo(self, x, y):
        pass
    
    def spam(self, x, *, z):
        pass
    
class B(A):
    def foo(self, a, b):
        pass
    
    def spam(self, x, z):
        pass

### 9.18 Defining Classes Programmatically

In [50]:
__name__

'__main__'

In [51]:
def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price
    
def cost(self):
    return self.shares * self.price

cls_dict = {
    '__init__': __init__,
    'cost': cost,
}

Stock = types.new_class('Stock', (), {}, lambda ns: ns.update(cls_dict))
Stock.__module__ = __name__

In [52]:
goog = Stock('GOOG', 2, 1000.)
goog.cost()

2000.0

### 9.19 Initializing Class Members at Definition Time

In [53]:
class StructTupleMeta(type):
    def __init__(cls, clsname, bases, clsdict):
        print('Meta __init__ called')
        super().__init__(clsname, bases, clsdict)
        for n, name in enumerate(cls._fields):
            print('Setting property {}'.format(name))
            setattr(cls, name, property(operator.itemgetter(n))) # Itemgetter works on instance self i.e., tuple
            
class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields = []
    def __new__(cls, *args):
        print('Class __new__ called')
        if len(args) != len(cls._fields):
            raise ValueError('{} arguments required.'.format(len(cls._fields)))
        return super().__new__(cls, args)

Meta __init__ called


In [54]:
class Stock(StructTuple):
    _fields = ['name', 'shares', 'price']

Meta __init__ called
Setting property name
Setting property shares
Setting property price


In [55]:
goog = Stock('GOOG', 2, 1000.)
print(goog)
print(goog[0])
print(goog.shares)
print(goog.shares * goog.price)

Class __new__ called
('GOOG', 2, 1000.0)
GOOG
2
2000.0


### 9.20 Multiple Dispatch

In [56]:
class MultiMethod:
    '''
    Represents a single multimethod
    '''
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name
        
    
    def register(self, meth):
        '''
        Register a new method as a multimethod
        '''
        sig = inspect.signature(meth)
        
        # Build a type signature from annotations
        types = []
        for name, parm in sig.parameters.items():
            if name == 'self':
                continue
            if parm.annotation is Parameter.empty:
                raise TypeError('Argument {} must be annotated with a type.'.format(name))
            if not isinstance(parm.annotation, type):
                raise TypeError('Argument {} annotation must be a type'.format(name))
            # Even if param with default value is not explicit in function call, the multimethod should trigger
            if parm.annotation is not Parameter.empty:
                self._methods[tuple(types)] = meth
            types.append(parm.annotation)
            
        self._methods[tuple(types)] = meth
        
    
    def __call__(self, *args):
        '''
        Call a method based on type signature
        '''
        types = tuple(type(arg) for arg in args[1:])
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            raise TypeError('Type signature not found')
            
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)
            

In [57]:
class MultiDict(dict):
    '''
    Special dict to build multimethods in a class
    '''
    def __setitem__(self, key, value):
        print('Setting key: {}'.format(key))
        if key in self:
            # If key already exists, it must be a multimethod or callable
            current_value = self[key]
            if isinstance(current_value, MultiMethod):
                current_value.register(value)
            else:
                # move callable into a new multimethod
                mvalue = MultiMethod(key) # new multimethod
                mvalue.register(current_value) # move old callable inside multimethod
                mvalue.register(value) 
                super().__setitem__(key, mvalue)
        else:
            super().__setitem__(key, value)
                

In [58]:
class MultipleMeta(type):
    '''
    Metaclass that allows multiple dispatch of methods
    '''
    def __new__(metacls, clsname, bases, clsdict):
        print('Meta __new__ called.')
        return super().__new__(metacls, clsname, bases, clsdict)
    
    @classmethod
    def __prepare__(metacls, clsname, bases):
        print('Meta __prepare__ called.')
        return MultiDict()

In [59]:
class Spam(metaclass = MultipleMeta):
    def bar(self, x:int, y:int):
        print('Bar 1: ', x, y)
    def bar(self, s:str, n:int=0):
        print('Bar 2: ', s, n)

Meta __prepare__ called.
Setting key: __module__
Setting key: __qualname__
Setting key: bar
Setting key: bar
Meta __new__ called.


In [60]:
Spam.__dict__

mappingproxy({'__module__': '__main__',
              'bar': <__main__.MultiMethod at 0x10d6c0748>,
              '__dict__': <attribute '__dict__' of 'Spam' objects>,
              '__weakref__': <attribute '__weakref__' of 'Spam' objects>,
              '__doc__': None})

In [61]:
s = Spam()
s.bar(2, 3)
s.bar('author', 0)
try:
    s.bar('author', 'Guido')
except TypeError as err:
    print(err)

Bar 1:  2 3
Bar 2:  author 0
Type signature not found


### 9.21 Repetitive Property Methods

In [62]:
def typed_property(name, expected_type):
    storage_name = '_'+name
    
    @property
    def prop(self):
        return getattr(self, storage_name)
    
    @prop.setter
    def prop(self, value):
        if not isinstance(value, expected_type):
            raise TypeError('{} must be a {}'.format(name, expected_type))
        setattr(self, storage_name, value)
    
    return prop

# Example use
class Person:
    name = typed_property('name', str)
    age = typed_property('age', int)
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [63]:
guido = Person('Guido', 1000)
print(guido.name, guido.age)
try:
    Person(0, 0)
except TypeError as err:
    print(err)


Guido 1000
name must be a <class 'str'>


In [64]:
String = partial(typed_property, expected_type=str)

class Student:
    name = String('name')
    def __init__(self, name):
        self.name = name

In [65]:
s = Student('Alex')
s.name
try:
    Student(0)
except TypeError as err:
    print(err)

name must be a <class 'str'>


### 9.22 Context managers

In [66]:
from contextlib import contextmanager

In [67]:
@contextmanager
def mycontext():
    print('Enter')
    try:
        yield 'contextobj'
    except RuntimeError as err:
        print(err)
    print('Exit')

In [68]:
with mycontext() as ret:
    print('User')
    print(ret)
    raise RuntimeError('oops')

Enter
User
contextobj
oops
Exit
